## Fine-tuning Phi-3-mini-QLoRA

Based on Phi-3Cookbook https://github.com/microsoft/Phi-3CookBook/blob/main/code/04.Finetuning/Phi-3-finetune-qlora-python.ipynb

Install required packages

In [16]:
# !pip install -qqq --upgrade bitsandbytes transformers peft accelerate datasets trl flash_attn
# !pip install huggingface_hub
# !pip install python-dotenv
# !pip install wandb -qqq
# !pip install absl-py nltk rouge_score
# !pip list | grep transformers.

Import packages

In [17]:
import torch
from random import randrange
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [18]:
model_name = 'acorreal/phi3-mental-health'
adapter_name = 'acorreal/adapter-phi-3-mini-mental-health'
compute_dtype = torch.bfloat16

In [19]:
%%time

# Load model
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, torch_dtype=compute_dtype)
model = PeftModel.from_pretrained(model, adapter_name)
model = model.merge_and_unload()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(adapter_name)

# Print model name
print("Model:", model.name_or_path)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
d:\fine-tune\.venv\Lib\site-packages\peft\tuners\lora\bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model: acorreal/phi3-mental-health
CPU times: total: 5.33 s
Wall time: 1.95 s


In [20]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [21]:
# Test the template
pipe.tokenizer.apply_chat_template([
    {
        "role": "user",
        "content": "Hello, I am stressed"}
    ],
    tokenize=False,
    add_generation_prompt=True
)

'<s><|user|>\nHello, I am stressed<|end|>\n<|assistant|>\n'

In [22]:
def predict(prompt: str) -> str:
    prompt = pipe.tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, num_beams=1, temperature=0.3, top_k=50, top_p=0.95, max_time= 180)
    print(outputs)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [23]:
%%time
predict("i am going through some things with my feelings and myself i barely sleep and i do nothing but think about how i am worthless and how i should not be here i have never tried or contemplated suicide i have always wanted to fix my issues but i never get around to it how can i change my feeling of being worthless to everyone")

[{'generated_text': "<s><|user|>\ni am going through some things with my feelings and myself i barely sleep and i do nothing but think about how i am worthless and how i should not be here i have never tried or contemplated suicide i have always wanted to fix my issues but i never get around to it how can i change my feeling of being worthless to everyone<|end|>\n<|assistant|>\n It's really brave of you to reach out and share your feelings, and I want to assure you that you are indeed valuable and worthy of love and respect. The thoughts you're experiencing are a sign that you're going through a tough time, but they don't define you. Here are some steps you can take to start changing those feelings and improve your mental health:\n\n1. Seek professional help: Reach out to a therapist or counselor who can help you work through your feelings of worthlessness and develop coping strategies.\n\n2. Practice self-care: Engage in activities that bring you joy and relaxation, such as exercise, 

"It's really brave of you to reach out and share your feelings, and I want to assure you that you are indeed valuable and worthy of love and respect. The thoughts you're experiencing are a sign that you're going through a tough time, but they don't define you. Here are some steps you can take to start changing those feelings and improve your mental health:\n\n1. Seek professional help: Reach out to a therapist or counselor who can help you work through your feelings of worthlessness and develop coping strategies.\n\n2. Practice self-care: Engage in activities that bring you joy and relaxation, such as exercise, meditation, or hobbies. Prioritize your physical and mental well-being.\n\n3. Build a support system: Surround yourself with friends and family who care about you and can provide emotional support.\n\n4. Set small, achievable goals: Break down your larger goals into smaller, more manageable tasks. Celebrate your accomplishments, no matter how small.\n\n5. Practice self-compassio